# Import libraries

In [1]:
import random
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer

from surprise import Dataset, Reader, SVD, KNNBasic, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.accuracy import rmse

# Load data

In [2]:
df_business_data = pd.read_json("yelp_academic_dataset_business.json", lines=True)

file_path = 'yelp_academic_dataset_review.json'

chunk_size = 100000
chunks = []
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    chunks.append(chunk)

df_review_data = pd.concat(chunks, ignore_index=True)

# Obtain negative samples

### Create a user id and business id pairs

In [3]:
unique_users = df_review_data['user_id'].unique()
unique_businesses = df_review_data['business_id'].unique()

business_user_reviewed_set = set(zip(df_review_data['user_id'], df_review_data['business_id'])) 
print((list(business_user_reviewed_set)[:10]))

[('Quww99MxkPPxU1iKmo2QZg', 'yz0KWVamNhqiZGz7XTClzg'), ('bevShY7Afipnm5OIZFY0PA', 'ZPrw52pIvcF3walP8nwXJw'), ('YSR5vcmt6kK4f41PBqfHNQ', 'PVieWiYn8GK9GSLnNFmOyA'), ('8D2xhwdMevU_KEpAf3IL7g', '1y3y_ihpO6SHxK9MRdtSlw'), ('6kvgjVvkG4dCE7g5fuzWwA', 'RmShOF-8MxNjlXATWs_2Dg'), ('lGpa9f6untBUjuJKrS4_TQ', 'SuSEmi52lP8gquHV0XIB9g'), ('mcSWsELdXF7HlLJfhYpWHQ', 'I3rlaQ_6ZTXsS3ZMh6LR2w'), ('_LT9U6nhRgny5folsaTxPA', 'xT41eqUSsM_MDE1Vjhq20A'), ('Ry2eEdDFbB3jCd-puLq0bg', '4SRl5od0lY33oK1GMbA23A'), ('Ca1tVnsBXdrWwWyt_AGbCg', '6_T2xzR74JqGCTPefAD8Tw')]


### Get positive samples

In [4]:
train_data = pd.DataFrame(list(business_user_reviewed_set), columns=["user_id", "business_id"])
train_data["reviewed"] = 1
train_data.head()

,user_id,business_id,reviewed
0,Quww99MxkPPxU1iKmo2QZg,yz0KWVamNhqiZGz7XTClzg,1
1,bevShY7Afipnm5OIZFY0PA,ZPrw52pIvcF3walP8nwXJw,1
2,YSR5vcmt6kK4f41PBqfHNQ,PVieWiYn8GK9GSLnNFmOyA,1
3,8D2xhwdMevU_KEpAf3IL7g,1y3y_ihpO6SHxK9MRdtSlw,1
4,6kvgjVvkG4dCE7g5fuzWwA,RmShOF-8MxNjlXATWs_2Dg,1


### Obtain negative samples

In [5]:
negative_samples = []
for user in unique_users[:len(train_data)]:  
    random_business = random.choice(unique_businesses)
    while (user, random_business) in business_user_reviewed_set:
        random_business = random.choice(unique_businesses)
    negative_samples.append((user, random_business, 0))

### Combine samples and save to CSV

In [6]:
review_prediction_data = pd.concat([
    train_data,
    pd.DataFrame(negative_samples, columns=["user_id", "business_id", "reviewed"])
])

review_prediction_data.to_csv("truedataset.csv", index=False)

### Sample data to predict

In [7]:
data_review = review_prediction_data.sample(frac=0.2, random_state=42)[["user_id", "business_id"]]
data_review.to_csv("review.csv", index=False)

# Models

## SVD - Binary

### Load and split data

In [8]:
reader = Reader(rating_scale=(0, 1)) # 0 reviewed , 1 not reviewed
surprise_data = Dataset.load_from_df(review_prediction_data[["user_id", "business_id", "reviewed"]], reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2)

### Fit SVD

model = SVD()
model.fit(trainset)
predictions = model.test(testset)

rmse(predictions)

### Tune params

In [ ]:
param_grid = {
    "n_factors": [25],
    "n_epochs": [20],
    "lr_all": [0.005],
    "reg_all": [0.02]
}

grid_search = GridSearchCV(SVD, param_grid, measures = ["rmse"], cv = 3)
grid_search.fit(surprise_data)

best_param = grid_search.best_params["rmse"]

print("Best Params:", best_param)

best_model = SVD(**best_param)
best_model.fit(trainset)

predictions = best_model.test(testset)

rmse(predictions)

Best Params: {'n_factors': 25, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


### Save prediction

In [ ]:
prediction_svd_df = pd.DataFrame([(p.uid, p.iid, p.est) for p in predictions], columns=["user_id", "business_id", "predicted"])
prediction_svd_df.to_csv("predictions_review_svd.csv", index=False)

## KNN - Binary

### Load and split data

In [11]:
review_prediction_data = pd.read_csv('truedataset.csv')
filtered_df = review_prediction_data.groupby("user_id").filter(lambda x: len(x) > 1)

small_df = filtered_df.sample(n=30000, random_state=42) 

reader = Reader(rating_scale=(0, 1))
surprise_data = Dataset.load_from_df(small_df[["user_id", "business_id", "reviewed"]], reader)
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

### Fit KNN

sim_options = {"name": "cosine", "user_based": True, "k": 5,"min_support": 2}  
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
predictions = model.test(testset)

rmse = accuracy.rmse(predictions)
print("RMSE:", rmse)

### Tune params

In [12]:
param_grid = {
    "name": ["cosine"],
    "user_based": [True],
    "k": [5],
    "min_support": [1]
}

grid_search = GridSearchCV(KNNBasic, param_grid, measures = ["rmse"], cv = 3)
grid_search.fit(surprise_data)

best_param = grid_search.best_params["rmse"]

print("Best Params:", best_param)

best_model = KNNBasic(**best_param)
best_model.fit(trainset)

predictions = best_model.test(testset)

rmse = accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Best Params: {'name': 'cosine', 'user_based': True, 'k': 5, 'min_support': 1}
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.4151


### Save predictions

In [13]:
predictions_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], 
                              columns=["user_id", "business_id", "predicted_reviewed"])
predictions_df.to_csv("knn_basic_review_predictions_small.csv", index=False)
print("Predictions saved to knn_basic_review_predictions_small.csv")

Predictions saved to knn_basic_review_predictions_small.csv


## SVD - Rating

### Load and split data

In [14]:
train_ratings = df_review_data[["user_id", "business_id", "stars"]]
test_ratings = train_ratings.sample(frac=0.2, random_state=42)[["user_id", "business_id"]]
test_ratings.to_csv("rating.csv", index=False)

review_data = train_ratings

reader = Reader(rating_scale=(1, 5)) 
data = Dataset.load_from_df(review_data[["user_id", "business_id", "stars"]], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Fit SVD

model = SVD()
model.fit(trainset)
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print("RMSE:", rmse)

### Tune params

In [15]:
param_grid = {
    "n_factors": [25],
    "n_epochs": [20],
    "lr_all": [0.005],
    "reg_all": [0.02]
}

grid_search = GridSearchCV(SVD, param_grid, measures = ["rmse"], cv = 3)
grid_search.fit(data)

best_param = grid_search.best_params["rmse"]

print("Best Params:", best_param)

best_model = SVD(**best_param)
best_model.fit(trainset)

predictions = best_model.test(testset)

rmse = accuracy.rmse(predictions)

Best Params: {'n_factors': 25, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
RMSE: 1.2790


### Save predictions

In [16]:
predictions_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], 
                              columns=["user_id", "business_id", "predicted_stars"])
predictions_df.to_csv("svd_rating_predictions.csv", index=False)

## KNN - Rating

### Load and split data

In [17]:
filtered_df = review_data.groupby("user_id").filter(lambda x: len(x) > 1)
small_df = filtered_df.sample(n=5000, random_state=42)
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(small_df[["user_id", "business_id", "stars"]], reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

### Fit KNN

sim_options = {"name": "cosine", "user_based": True, "k": 5, "min_support": 2}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
print("RMSE:", rmse)

### Tune params

In [18]:
param_grid = {
    "name": ["cosine"],
    "user_based": [True],
    "k": [5],
    "min_support": [1]
}

grid_search = GridSearchCV(KNNBasic, param_grid, measures = ["rmse"], cv = 3)
grid_search.fit(surprise_data)

best_param_ = grid_search.best_params["rmse"]

print("Best Params:", best_param_)

best_model = KNNBasic(**best_param_)
best_model.fit(trainset)

predictions = best_model.test(testset)

rmse = accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Best Params: {'name': 'cosine', 'user_based': True, 'k': 5, 'min_support': 1}
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4651


### Save predictions

In [19]:
predictions_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], 
                              columns=["user_id", "business_id", "predicted_rating"])
predictions_df.to_csv("knn_rating_predictions.csv", index=False)

## Category Analysis

In [20]:
df_business_data["categories"] = df_business_data["categories"].fillna("")
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(", "), lowercase=False)
tfidf_matrix = vectorizer.fit_transform(df_business_data["categories"])
category_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=category_names, index=df_business_data.index)
df_business_data["minimized_category"] = tfidf_df.idxmax(axis=1)

#print(df_business_data[["business_id", "categories", "minimized_category"]])
category_counts = df_business_data["minimized_category"].value_counts()
print(category_counts)

/opt/conda/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


minimized_category
Burgers           2375
Nail Salons       2083
Pizza             2009
Italian           1918
Mexican           1872
                  ... 
Cucina campana       1
Dart Arenas          1
Circus Schools       1
Bubble Soccer        1
Trade Fairs          1
Name: count, Length: 1305, dtype: int64


## Final SVD Model

### Load and split data

In [21]:
df_review_data = df_review_data[["user_id", "business_id", "stars"]]
df_business_data = df_business_data[["business_id", "minimized_category", "name"]]
df_review_data = df_review_data.merge(df_business_data, on="business_id")

reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(df_review_data[["user_id", "business_id", "stars"]], reader)
trainset, testset = train_test_split(surprise_data, test_size=0.2)

### Fit final SVD with best params

In [ ]:
model = SVD(**best_param)
model.fit(trainset)

predictions = model.test(testset)
predictions_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions], 
                              columns=["user_id", "business_id", "predicted_rating"])

predictions_df = predictions_df.merge(df_business_data, on="business_id")

### Generate recommendation

In [ ]:
N = 5
predictions_df = predictions_df.sort_values(["user_id", "predicted_rating"], ascending=[True, False])
top_n_recommendations = predictions_df.groupby("user_id").head(N)

italian_reviews = df_review_data[df_review_data["minimized_category"] == "Italian"]
user_italian_counts = italian_reviews.groupby("user_id").size().reset_index(name="count")
low_review_italian_users = user_italian_counts[user_italian_counts["count"] <= 3]["user_id"]

filtered_recommendations = top_n_recommendations[
    (top_n_recommendations["user_id"].isin(low_review_italian_users)) &
    (top_n_recommendations["minimized_category"] == "Italian")
]

### Evaluate

In [ ]:
actual_italian = df_review_data[df_review_data["minimized_category"] == "Italian"]
actual_italian_users = actual_italian.groupby("user_id")["name"].apply(set).reset_index()
recommended_italian_users = filtered_recommendations.groupby("user_id")["name"].apply(set).reset_index()

merged = actual_italian_users.merge(recommended_italian_users, on="user_id", how="outer", suffixes=("_actual", "_predicted"))
merged["name_actual"] = merged["name_actual"].apply(lambda x: x if isinstance(x, set) else set())
merged["name_predicted"] = merged["name_predicted"].apply(lambda x: x if isinstance(x, set) else set())

merged["true_positives"] = merged.apply(lambda x: len(x["name_actual"] & x["name_predicted"]), axis=1)
merged["precision"] = merged["true_positives"] / merged["name_predicted"].apply(lambda x: max(1, len(x)))
merged["recall"] = merged["true_positives"] / merged["name_actual"].apply(lambda x: max(1, len(x)))

avg_precision = merged["precision"].mean()
avg_recall = merged["recall"].mean()
hit_rate = (merged["true_positives"] > 0).mean()

print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"Hit Rate: {hit_rate:.4f}")

global_top_italian = df_review_data[df_review_data["minimized_category"] == "Italian"].groupby("name")["stars"].mean().reset_index()
global_top_italian = global_top_italian.sort_values("stars", ascending=False).head(5)  # Take top 5 highest-rated Italian places

def get_fallback_recommendation():
    return np.random.choice(global_top_italian["name"].values)

for idx, row in merged.head(10).iterrows():
    recommended_names = row["name_predicted"] if row["name_predicted"] else {get_fallback_recommendation()}
    print(f"User ID: {row['user_id']}")
    print(f"Reviewed Italian Restaurants: {row['name_actual']}")
    print(f"Recommended Italian Restaurants: {recommended_names}")
    print("-" * 50)